In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from tqdm import tqdm
from torch.nn import Conv2d, BatchNorm2d,Dropout,Flatten,MaxPool2d,Linear
from torch.nn import ReLU,Sigmoid,Softmax
from torch.nn import Sequential

In [2]:
class ObjectDetection(nn.Module):
    def __init__(self,n_class):
        super(ObjectDetection,self).__init__()
        self.n_class=n_class
        self.base_model=Sequential(
            Conv2d(in_channels=3,out_channels=64,kernel_size=3,padding='same',stride=1),
            BatchNorm2d(num_features=64),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2,stride=2),
            Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding='same',stride=1),
            BatchNorm2d(num_features=128),
            ReLU(inplace=True),
            Conv2d(in_channels=128,out_channels=128,kernel_size=3,padding='same',stride=1),
            BatchNorm2d(num_features=128)
        )
        
        self.regressor = Sequential(
    		Linear(in_features=128, out_features=128),
    		ReLU(inplace=True),
    		Linear(in_features=128, out_features=64),
    		ReLU(inplace=True),
    		Linear(in_features=64, out_features=32),
    		ReLU(inplace=True),
    		Linear(in_features=32, out_features=4),
    		Sigmoid()
        )
        
        self.classifier = Sequential(
			Linear(in_features=512, out_features=512),
			ReLU(inplace=True),
			Dropout(0.2),
			Linear(in_features=512, out_features=512),
			ReLU(inplace=True),
			Dropout(0.2),
			Linear(in_features=512, out_features=self.n_class)
        )

    def farward(self,inputs):
        x=self.base_model(inputs)
        x=x.view(x.size(0),-1)
        regressor=self.regressor(x)
        classier=self.classifier(x)
        return (regressor,classifier)


In [4]:
model=ObjectDetection(n_class=10)
model

ObjectDetection(
  (base_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (regressor): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features